In [1]:
%env DATA_BUCKET=tbd-2023z-9910-data
%env GEN_OUTPUT_DIR=/tmp/tpc-di
%env REPO_ROOT=/home/jupyter/git/tbd-tpc-di/

env: DATA_BUCKET=tbd-2023z-9910-data
env: GEN_OUTPUT_DIR=/tmp/tpc-di
env: REPO_ROOT=/home/jupyter/git/tbd-tpc-di/


In [ ]:
!pip3.8 install typer[All]==0.9.0 google-cloud-storage==2.13.0

## Install SDKMAN for setting up JVM 8 enviroment

In [ ]:
%%bash
curl -s https://get.sdkman.io | bash

## Install and set as default JVM 8

In [ ]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
sdk install java 8.0.392-amzn
sdk use java 8.0.392-amzn

## Check if JVM 8 is used

In [ ]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
java -version

## Clone tbd-tpc-di repo

In [ ]:
%%bash
mkdir -p git && cd git
git clone https://github.com/mwiewior/tbd-tpc-di.git
cd tbd-tpc-di
git checkout feat/labs
git pull

## Generate input dataset (run this cell below from the terminal!!!)
It should take approx. 15min with scale factor set to 100 and generate approx. 10GiB

In [ ]:
%%script bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
cd /home/jupyter/git/tbd-tpc-di/tools/ 
java -jar DIGen.jar -sf 100 -o /tmp/tpc-di

In [ ]:
# Install and setup JVM 11

In [ ]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
sdk install java 11.0.21-amzn
sdk use java 11.0.21-amzn -y

## Load staging

In [ ]:
%%bash
source "$HOME/.sdkman/bin/sdkman-init.sh"
cd $REPO_ROOT
python3.8 tpcdi.py --output-directory $GEN_OUTPUT_DIR --stage $DATA_BUCKET

## Setup dbt

In [ ]:
%%bash
cd $REPO_ROOT
pip3.8 install -r requirements.txt 

## Test dbt connection

In [ ]:
%%bash
cd $REPO_ROOT
dbt debug

## Install dbt deps

In [ ]:
%%bash
cd $REPO_ROOT
dbt deps

## Run dbt ELT

In [ ]:
%%bash
cd $REPO_ROOT
dbt run

11:42:04  Running with dbt=1.7.3
11:42:05  Registered adapter: spark=1.7.1
11:42:06  Found 45 models, 1 test, 17 sources, 0 exposures, 0 metrics, 553 macros, 0 groups, 0 semantic models
11:42:06  
:: loading settings :: url = jar:file:/usr/local/lib/python3.8/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2c86d8ea-06c9-4950-a58a-c8d01780922e;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.17.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
:: resolution report :: resolve 571ms :: artifacts dl 23ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.17.0 from central in [default]
	commons-io#commons-io;2.11.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.3.0 from central in [default]
	org.glassfish.jaxb#txw2;3.0.2 from central in [default]
	org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 from central in [default]
	----------------------

23/12/04 11:42:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/04 11:42:16 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
23/12/04 11:42:16 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
23/12/04 11:42:24 WARN Client: Same path resource file:///root/.ivy2/jars/com.databricks_spark-xml_2.12-0.17.0.jar added multiple times to distributed cache.
23/12/04 11:42:24 WARN Client: Same path resource file:///root/.ivy2/jars/commons-io_commons-io-2.11.0.jar added multiple times to distributed cache.
23/12/04 11:42:24 WARN Client: Same path resource file:///root/.ivy2/jars/org.glassfish.jaxb_txw2-3.0.2.jar added multiple times to distributed cache.
23/12/04 11:42:24 WARN Client: Same path resource file:///root/.ivy2/jars/org.apache.ws.xmlschema_xmlschema-core-2.3.0.jar added multiple times to distributed cache.
23/12/04 11:42:24 WARN Client: Same path resource file:///root/.ivy2/jars/org.scala-lang.modules_scala

## Run dbt tests

In [ ]:
%%bash
cd $REPO_ROOT
dbt test

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TBD-TPC-DI-setup") \
    .master("yarn") \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
spark.sql("show databases").show()

In [ ]:
spark.sql("use demp_gold")

In [ ]:
spark.sql("show tables").show()

In [ ]:
spark.stop()